# YOLOv8 Model for Medical Masks Detection

### Environment and GPU Check

In [6]:
import os
import re
import time
import yaml
import shutil

import torch
import torch.onnx
from ultralytics.models import YOLO

import wandb
from roboflow import Roboflow
from wandb.integration.ultralytics import add_wandb_callback


!nvidia-smi
print("Cuda available") if torch.cuda.is_available() else print("Cuda is not available")
HOME = 'C:/Users/kacpi/PycharmProjects/Medical-Mask-Presence-Detection'

Wed Nov  1 16:57:41 2023       Cuda available

+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.58                 Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              14W /  85W |    115MiB /  6144MiB |     44%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------

In [7]:
wandb.login()

True

### Parameters

In [8]:
PROJECT = "Medical-Mask-Presence-Detection"
VERSION = 5
EPOCHS = 40
BS = 64
IMG_SIZE = 640

### Dataset Preparation
Backup the original data.yaml and then download the dataset from Roboflow.

In [9]:
version_name = f'Mask-Detection-YOLOv8-{VERSION}'
full_dataset_path = os.path.join(HOME, 'YOLO', version_name)

if os.path.exists(full_dataset_path):
    print(f"{version_name} already exists in the directory!")
    dataset_location = full_dataset_path
else:
# Download dataset from Roboflow
    with open('apiKey.txt') as api:
        rf = Roboflow(api_key=api.read())
        project = rf.workspace("agh-ett2f").project("mask-detection-yolov8")
        dataset = project.version(VERSION).download("yolov8")
        dataset_location = dataset.location

    shutil.copy('Mask-Detection-YOLOv8-1/data_backup.yaml', f'Mask-Detection-YOLOv8-{VERSION}/data.yaml')

print("Dataset location:", dataset_location)

Mask-Detection-YOLOv8-5 already exists in the directory!
Dataset location: C:/Users/kacpi/PycharmProjects/Medical-Mask-Presence-Detection\YOLO\Mask-Detection-YOLOv8-5


### Training the Model
Define parameters and train the model using the downloaded dataset.

In [10]:
# Initialize the YOLOv8 model
model = YOLO(f"yolov8s.pt")

# Set up Weights and Biases for experiment tracking
run = wandb.init(project=PROJECT, config={"epochs": EPOCHS, "batch_size": BS})
add_wandb_callback(model, enable_model_checkpointing=True)

# Training the model
start_time = time.time()
model.train(project=PROJECT ,data=f'{dataset_location}/data.yaml', epochs=EPOCHS,  imgsz=IMG_SIZE)
end_time = time.time() - start_time

print(f"Training took {end_time/60:.2f} minutes ({end_time:.2f} seconds)")

New https://pypi.org/project/ultralytics/8.0.203 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.186  Python-3.11.3 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:/Users/kacpi/PycharmProjects/Medical-Mask-Presence-Detection\YOLO\Mask-Detection-YOLOv8-5/data.yaml, epochs=40, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=Medical-Mask-Presence-Detection, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=Fal

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\kacpi\\AppData\\Local\\Temp\\tmpoma8h6r4wandb-media\\h8wr1t8p.table.json'

### Validation and Testing
After training, validate and test the model using the appropriate datasets.

In [13]:
# Helper function to retrieve the directory with the latest training
def train_dir_with_biggest_number():
    directory = f'{HOME}/YOLO/Medical-Mask-Presence-Detection/'
    folders = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    pattern = r'^train(\d+)$'
    max_num = -1
    max_folder = None

    for folder in folders:
        match = re.match(pattern, folder)
        if match:
            num = int(match.group(1))
            if num > max_num:
                max_num = num
                max_folder = folder
        else:
            max_folder = "train"

    return max_folder

max_train = train_dir_with_biggest_number()
print("Saved to: " + str(max_train))

trainedModelPath = f'{HOME}/YOLO/Medical-Mask-Presence-Detection/{max_train}/weights/best.pt'
trainedModelDir=f'Medical-Mask-Presence-Detection/train/'
trainedModel = YOLO(trainedModelPath)

Saved to: train3


In [14]:
# # Validate the model
trainedModel.val(data=f'{dataset_location}/data.yaml')

# Predict using the trained model on test images
trainedModel.predict(conf=0.25, source=f'{dataset.location}/test/images')

Ultralytics YOLOv8.0.186  Python-3.11.3 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning C:\Users\kacpi\PycharmProjects\Medical-Mask-Presence-Detection\YOLO\Mask-Detection-YOLOv8-5\valid\labels.cache... 302 images, 0 backgrounds, 0 corrupt: 100%|██████████| 302/302 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:10<00:00,  1.88it/s]
                   all        302       1039      0.924      0.799      0.869      0.598
 mask_weared_incorrect        302        462      0.958      0.889      0.952      0.698
             with_mask        302        577      0.889      0.709      0.786      0.498
Speed: 0.8ms preprocess, 22.6ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to C:\Users\kacpi\PycharmProjects\Medical-Mask-Presence-Detection\runs\detect\val4

image 1/295 C:

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'mask_weared_incorrect', 1: 'with_mask', 2: 'without_mask'}
 orig_img: array([[[62, 62, 62],
         [62, 62, 62],
         [62, 62, 62],
         ...,
         [67, 67, 67],
         [67, 67, 67],
         [67, 67, 67]],
 
        [[63, 63, 63],
         [62, 62, 62],
         [62, 62, 62],
         ...,
         [68, 68, 68],
         [68, 68, 68],
         [68, 68, 68]],
 
        [[63, 63, 63],
         [63, 63, 63],
         [63, 63, 63],
         ...,
         [67, 67, 67],
         [67, 67, 67],
         [67, 67, 67]],
 
        ...,
 
        [[13, 13, 13],
         [14, 14, 14],
         [16, 16, 16],
         ...,
         [33, 33, 33],
         [35, 35, 35],
         [35, 35, 35]],
 
        [[10, 10, 10],
         [11, 11, 11],
         [14, 14, 14],
         ...,
         [32, 32, 32],
         [33, 33, 33],
         [34, 

In [15]:
# Deploying the model for inference
project.version(VERSION).deploy(model_type='yolov8', model_path=trainedModelDir)

Dependency ultralytics==8.0.134 is required but found version=8.0.186, to fix: `pip install ultralytics==8.0.134`
View the status of your deployment at: https://app.roboflow.com/agh-ett2f/mask-detection-yolov8/5
Share your model with the world at: https://universe.roboflow.com/agh-ett2f/mask-detection-yolov8/model/5


In [16]:
# Load the YOLOv8 model with trained weights
model = torch.load(trainedModelPath)['model'].float().eval()

dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE)

# Export to ONNX
onnx_path = 'yolov8_custom.onnx'
torch.onnx.export(model, dummy_input, onnx_path, verbose=True, opset_version=11)

Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).


In [17]:
# Clear GPU memory for optimization (import needed)
import torch
torch.cuda.empty_cache()